<a href="https://colab.research.google.com/github/ShivamThapa243/Information-Retrieval/blob/main/positional_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREPROCESSING STAGE

**Building a prositional inverted index**

In [2]:
# connecting to google drive to access the preprocessed data set

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Function to build an positional inverted index**

Structure of positional inverted index:

    { word_1:
      [ document_count : x,
        { document_id_1 : [position_1, position_2, position_3] },
        { document_id_2 : [position_4, position_5, position_6] },
        { document_id_3 : [position_7, position_8, position_9] }
      ]
    },
    { word_2:
      [ document_count : y,
        { document_id_1 : [position_1, position_2, position_3] },
        { document_id_12 : [position_4, position_5, position_6] },
        { document_id_23 : [position_7, position_8, position_9] }
      ]
    } ...